In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import datetime
import time

In [0]:
dbfs_mount_path               = "dbfs:/mnt/marketoimportexport"
csv_folderpath                = "%s/suppressionscore/" % dbfs_mount_path
file_generated_timestamp      = datetime.datetime.now().strftime('%Y%m%d%H%M')
dateUpdated_timestamp        = datetime.datetime.now().strftime('%Y%m%d %H:%M:%S')
export_filename               = "SuppressionScores_" + str(file_generated_timestamp) 

print ('DBFS File path : ' , csv_folderpath)
print(export_filename)
print(dateUpdated_timestamp)

DBFS File path : dbfs:/mnt/marketoimportexport/suppressionscore/
SuppressionScores_202010061431
20201006 14:31:12

In [0]:
ufs_scores       = spark.sql("""show tables in data_suppression""")
ufs_scores       = ufs_scores.where(col("tableName").rlike("perfect_golden_ufs_recipientscores"))
ufs_scores       = ufs_scores.withColumn("date", substring(col("tableName"),36,100))

window           = Window.partitionBy('database').orderBy(col('date').desc())
ufs_scores       = ufs_scores.withColumn("rank",rank().over(window))
ufs_scores       = ufs_scores.where(col("rank") == 1)
table_name       = "data_suppression." + ufs_scores.select("tableName").rdd.max().tableName
print((table_name))

latest_suppression_scores = spark.table(table_name)
latest_suppression_scores = latest_suppression_scores.drop("countryCode")

data_suppression.perfect_golden_ufs_recipientscores_202010061249

In [0]:
latest_suppression_scores = latest_suppression_scores.withColumnRenamed("cp_countryCode", "countryCode")
latest_suppression_scores = latest_suppression_scores.withColumnRenamed("cp_ohubId", "contactOhubId")
latest_suppression_scores = latest_suppression_scores.withColumnRenamed("Score", "score")
latest_suppression_scores = latest_suppression_scores.withColumn("dateUpdated", lit(dateUpdated_timestamp))
latest_suppression_scores = latest_suppression_scores.select("countryCode","contactOhubId","score","dateUpdated").distinct()

In [0]:
%run "/Shared/dataPipelines/OperatorLifeCycle/S01_Libraries_Functions"

In [0]:
latest_suppression_scores.repartition(1).write.option("sep",",").csv(csv_folderpath + export_filename +".csv" , mode="overwrite" , header=True)
copyMerge(csv_folderpath + export_filename +".csv", csv_folderpath + export_filename + "_merged.csv", debug=True, overwrite=True, deleteSource=True)

Appending file dbfs:/mnt/marketoimportexport/suppressionscore/SuppressionScores_202010061431.csv/part-00000-tid-978133212727993233-ae0dcef5-f0ce-45cf-ab27-428f777712a1-2483534-1-c000.csv into dbfs:/mnt/marketoimportexport/suppressionscore/SuppressionScores_202010061431_merged.csv
Source directory dbfs:/mnt/marketoimportexport/suppressionscore/SuppressionScores_202010061431.csv removed.